In [13]:
import folium
import numpy as np
import io
import base64
from PIL import Image
import selenium.webdriver as webdriver
from selenium.webdriver.chrome.options import Options
import time
import os
import imageio

def create_drone_satellite_map_gif(output_filename="drone_map_animation.gif", 
                                 frames=36, 
                                 radius_km=5.0, 
                                 duration=0.2):
    """
    Creates an animated GIF of a satellite map showing St. Gallen with a drone circling around it.
    
    Args:
        output_filename (str): Name of the output GIF file
        frames (int): Number of frames in the animation (more frames = smoother)
        radius_km (float): Radius of the drone's circular path in kilometers
        duration (float): Duration between frames in seconds
    
    Returns:
        str: Path to the saved GIF file
    """
    
    # St. Gallen coordinates
    st_gallen_lat = 47.4245
    st_gallen_lon = 9.3767
    
    # Convert radius from km to degrees (approximate)
    radius_deg = radius_km / 111.0  # 1 degree ≈ 111 km
    
    # Set up Chrome options for headless browsing
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1200,800")
    
    images = []
    
    try:
        # Initialize webdriver
        driver = webdriver.Chrome(options=chrome_options)
        
        for i in range(frames):
            # Calculate drone position
            angle = 2 * np.pi * i / frames
            drone_lat = st_gallen_lat + radius_deg * np.cos(angle)
            drone_lon = st_gallen_lon + radius_deg * np.sin(angle)
            
            # Create folium map centered on St. Gallen
            m = folium.Map(
                location=[st_gallen_lat, st_gallen_lon],
                zoom_start=13,
                tiles='Esri.WorldImagery'  # Satellite tiles
            )
            
            # Add St. Gallen marker
            folium.Marker(
                [st_gallen_lat, st_gallen_lon],
                popup='St. Gallen, CH',
                tooltip='St. Gallen',
                icon=folium.Icon(color='red', icon='home')
            ).add_to(m)
            
            # Add drone marker
            folium.Marker(
                [drone_lat, drone_lon],
                popup='Drone',
                tooltip='Drone',
                icon=folium.Icon(color='blue', icon='plane')
            ).add_to(m)
            
            # Add circular path
            folium.Circle(
                [st_gallen_lat, st_gallen_lon],
                radius=radius_km * 1000,  # Convert km to meters
                color='yellow',
                weight=2,
                opacity=0.7,
                fill=False
            ).add_to(m)
            
            # Add line from St. Gallen to drone
            folium.PolyLine(
                [[st_gallen_lat, st_gallen_lon], [drone_lat, drone_lon]],
                color='green',
                weight=2,
                opacity=0.8
            ).add_to(m)
            
            # Save map as HTML
            html_file = f"temp_map_{i}.html"
            m.save(html_file)
            
            # Open the HTML file with webdriver
            driver.get(f"file://{os.path.abspath(html_file)}")
            time.sleep(2)  # Wait for map to load
            
            # Take screenshot
            png_data = driver.get_screenshot_as_png()
            
            # Convert to PIL Image
            img = Image.open(io.BytesIO(png_data))
            images.append(img)
            
            # Clean up temporary HTML file
            os.remove(html_file)
            
            print(f"Frame {i+1}/{frames} completed")
        
        # Save as GIF
        imageio.mimsave(
            output_filename,
            images,
            duration=duration,
            loop=0
        )
        
        print(f"Animation saved as {output_filename}")
        return output_filename
        
    except Exception as e:
        print(f"Error creating animation: {e}")
        return None
    
    finally:
        # Clean up
        try:
            driver.quit()
        except:
            pass
        
        # Clean up any remaining temp files
        for i in range(frames):
            temp_file = f"temp_map_{i}.html"
            if os.path.exists(temp_file):
                os.remove(temp_file)

# Alternative version using matplotlib for systems without selenium
def create_drone_satellite_map_gif_matplotlib(output_filename="drone_map_animation_simple.gif",
                                            frames=36,
                                            radius_km=5.0,
                                            duration=200):
    """
    Creates a simplified animated GIF using matplotlib (no satellite imagery).
    Use this if you don't have selenium/chromedriver installed.
    
    Args:
        output_filename (str): Name of the output GIF file
        frames (int): Number of frames in the animation
        radius_km (float): Radius of the drone's circular path in kilometers
        duration (int): Duration between frames in milliseconds
    
    Returns:
        str: Path to the saved GIF file
    """
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    
    # St. Gallen coordinates
    st_gallen_lat = 47.4245
    st_gallen_lon = 9.3767
    
    # Convert radius from km to degrees (approximate)
    radius_deg = radius_km / 111.0
    
    images = []
    
    for i in range(frames):
        # Calculate drone position
        angle = 2 * np.pi * i / frames
        drone_lat = st_gallen_lat + radius_deg * np.cos(angle)
        drone_lon = st_gallen_lon + radius_deg * np.sin(angle)
        
        # Create plot
        fig, ax = plt.subplots(figsize=(10, 8))
        ax.set_xlim(st_gallen_lon - radius_deg * 1.5, st_gallen_lon + radius_deg * 1.5)
        ax.set_ylim(st_gallen_lat - radius_deg * 1.5, st_gallen_lat + radius_deg * 1.5)
        
        # Plot circular path
        circle = patches.Circle((st_gallen_lon, st_gallen_lat), radius_deg, 
                              fill=False, color='yellow', linewidth=2)
        ax.add_patch(circle)
        
        # Plot St. Gallen
        ax.plot(st_gallen_lon, st_gallen_lat, 'ro', markersize=10, label='St. Gallen')
        
        # Plot drone
        ax.plot(drone_lon, drone_lat, 'b^', markersize=8, label='Drone')
        
        # Plot connection line
        ax.plot([st_gallen_lon, drone_lon], [st_gallen_lat, drone_lat], 'g--', alpha=0.7)
        
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        ax.set_title(f'Drone Circling St. Gallen - Frame {i+1}/{frames}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal')
        
        # Save frame to bytes
        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=100, bbox_inches='tight')
        buf.seek(0)
        img = Image.open(buf)
        images.append(img)
        plt.close()
    
    # Save as GIF
    imageio.mimsave(output_filename, images, duration=duration, loop=0)
    print(f"Animation saved as {output_filename}")
    return output_filename

if __name__ == "__main__":
    # Example usage - try the satellite version first, fall back to matplotlib version
    try:
        result = create_drone_satellite_map_gif("st_gallen_drone_satellite.gif")
        if result is None:
            print("Falling back to matplotlib version...")
            result = create_drone_satellite_map_gif_matplotlib("st_gallen_drone_simple.gif")
    except Exception as e:
        print(f"Satellite version failed: {e}")
        print("Using matplotlib version...")
        result = create_drone_satellite_map_gif_matplotlib("st_gallen_drone_simple.gif")

Frame 1/36 completed
Frame 2/36 completed
Frame 3/36 completed
Frame 4/36 completed
Frame 5/36 completed
Frame 6/36 completed
Frame 7/36 completed
Frame 8/36 completed
Frame 9/36 completed
Frame 10/36 completed
Frame 11/36 completed
Frame 12/36 completed
Frame 13/36 completed
Frame 14/36 completed
Frame 15/36 completed
Frame 16/36 completed
Frame 17/36 completed
Frame 18/36 completed
Frame 19/36 completed
Frame 20/36 completed
Frame 21/36 completed
Frame 22/36 completed
Frame 23/36 completed
Frame 24/36 completed
Frame 25/36 completed
Frame 26/36 completed
Frame 27/36 completed
Frame 28/36 completed
Frame 29/36 completed
Frame 30/36 completed
Frame 31/36 completed
Frame 32/36 completed
Frame 33/36 completed
Frame 34/36 completed
Frame 35/36 completed
Frame 36/36 completed
Animation saved as st_gallen_drone_satellite.gif
